# BOT PROJECT
#### NOTEBOOK AIM
This notebook initially acts as notebook / script 1 in the below BOT METHOD #1. See below for details.

### BOT METHOD #1
**Notebook / Script 1:**
1. Call in a websocket price stream from coinbase pro. 
2. Every new minute send close/open (seconds==0) price to a pickle file (dataframe or dictionary)
3. Set a high low price for the given minute and adjust it as the stream sets new records. The low / high value should be reset to the 1m candle open price each minute
4. Every time the high/low changes send the new price to a pickle file


**Notebook / Script 2:** 
1. Calls in the passed close/open price every minute to calculate and construct a supertrend dataframe 
2. So the above will be the close of the last candle and the open of the current candle
3. Also calls in the low high value passed by notebook 1. 
4. Intra-minute the low high will fluctuate as new records are set
5. Handles the connection to trade platform
6. Executes buy / sell orders on supertrend
7. Executes take profits / stops base on the latest high low values passes from the coinbase websocket

**Possible Problems**
1. The extra time required to pass new high lows from one script to the next could mean a loss of trade value
2. We don't know the problem of having two connections open at once - is this possible even from separate notebooks.
3. Need to solve how to update the supertrend with a dataframe updating every minute

In [ ]:
# !pip install rel

In [ ]:
# Import packages
import pandas as pd
import cbpro
from credscb import creds
import websocket, json
import time
from datetime import datetime, timedelta, timezone
import pandas_ta as ta
import rel

In [ ]:
# Set the socket endpoint - coinbase pro feed
socket = "wss://ws-feed.pro.coinbase.com"

In [ ]:
def on_open(ws):
    """
    This function is called from the websocket method and holds the required info for the ws request
    """
    print('the socket is opened')
    subscribe_msg = {
        'type': 'subscribe',
        'channels':[
            {'name': 'ticker',
             'product_ids':['BTC-USD']}
        ]
    }
    error_msg = {
        'type': "error",
        'message': "connection lost!"
    }
    ws.send(json.dumps(subscribe_msg))
    ws.send(json.dumps(error_msg))

In [ ]:
# BASIC ON MESSAGE FUNCTION - LET"S BUILD INCREMENTALLY AS THE WEBSOCKET WAS FAILING WITH THE PREFERRED ABOVE

high = float()
low = float()

# Set an empty Master dataframe to store new and complete 1m price data
prices = pd.DataFrame(columns=["time", "open", "high", "low", "close"])

high_low_set = False


def on_message(ws, message):
    """
    This function handles the data retrieved from the websocket. We store the message into a json dict and then we
    can manage the data
    """
    global prices
    global high
    global low
    global high_low_set

    
    data = json.loads(message)
    datetime_val = pd.to_datetime(data["time"])
    
    if datetime_val.second<2 and high_low_set==False:
        # This can trigger up to five times with multiple messages within the first second of new candle
        prices = prices.append({'time': datetime_val,
                                'open': data['price'], 
                                'high': data['price'],
                                'low': data['price'],
                                'close': data['price']
                               }, ignore_index=True)
        print(f"1m price added! {datetime.now()}")
        
        # Set the high low of the current (just started candle) to the first of message (second==0) open price
        high = data['price']
        low = data['price']
        
        # Set High_low_set to True
        high_low_set = True
    
        prices.tail().to_csv("prices_test.csv", index=False)
    
    ## ADD another IF statement here to check a second=0 data message was actually recieved. If not it will 
    ## Mess up the data continuity and also the supertrend calculations!!
    
    elif datetime_val.second==59:
        # Get the last streamed price within current minute and set the close price of the candle to this price!
        close = data['price']
        prices.iat[-1, 4]=close   
        high_low_set = False ## moved to next statement
        prices.tail().to_csv("prices_test.csv", index=False)
        

        #### WITH THIS ELIF STATEMENT WE DON"T CHECK HIGHS LOWS WHEN SECOND==59 We need to fix this
        #### We could add an if else in this elif or just change the high/low checks to there own if statement
     
    # Set a statement to reset the high_low_set back to False
#     elif datetime_val.second>10 and high_low_set == True:
#         high_low_set=False
        #### IF WE DON:T RECIEVE A second==59 message -  the high_low will not be reset. All we need to do 
        #### Here is say any second above second == 10 and the high_low gets reset!! Solved
        
        
    
    # Always check 
    elif data['price']>high:
        # Change the high price at our last data points
        high = data['price']
        prices.iat[-1, 2]=high
        # Update the csv
        prices.tail().to_csv("prices_test.csv", index=False)
    elif data['price']<low:
        # Change the low price at our last data points
        low = data['price']
        prices.iat[-1, 3] = low
        # Update the csv
        prices.tail().to_csv("prices_test.csv", index=False)
    

    if data["type"]=="error":
        print("we have an error")

In [ ]:
# Set the websocket request to a variable
ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message)

In [ ]:
# Call in the websocket feed
while True:
    try:
        ws.run_forever()
    except:
        time.sleep(5)

  

In [ ]:
pd.read_csv("prices_test.csv")

## SANDPIT

Look into error handling and re-connections!

In [ ]:
def on_test(ws, message):
    data = json.loads(message)
    if data["type"]=="error":
        print("we have an error")
    else:
        print(data["price"])
    

In [ ]:
# Set the websocket request to a variable
wsx = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_test)

In [ ]:
wsx.run_forever()

In [ ]:
pd.DataFrame({'open': 34, 
              'high': 43,
              'low': 54,
              'close': 65,
              'time': datetime.now()}, index=range(0,4))


#### REDUNDANT ON MESSAGE FUNCTION

In [ ]:
# Set an empty dataframe to store the prices
prices = pd.DataFrame(columns=["time", "open", "high", "low", "close"])
prices

In [ ]:
prices = prices.append({'time': datetime.now(),
                        'open': 43000, 
                        'high': 43000,
                        'low': 43000,
                        'close': 43000}, ignore_index=True)

In [ ]:
prices

In [ ]:
prices.iloc[-1]

In [ ]:
new_high = 43500
old_high = 43000

if new_high > old_high:
    print("yes")
    prices.iat[-1, 1]=new_high
elif new_low < old_low:
    print("yes")
    prices.iat[-1, 2]=new_low

In [ ]:
prices.iat[-1, 1]=19